## Personal Finance Tracker through PDF Statement Data Extraction + Post-Analysis

This notebook interprets bank statements from Scotiabank. It extracts data from bank statement PDFs, bins them into categories based on extracted metadata, and then produces insights and visualizations on financial health, spending habits and so on. It is intended to be used as a means of insights into spending, long-term growth and ways to improve financial health overall.

In [1]:
import sys
import os
sys.path.append(os.path.abspath('../..'))

In [2]:
from src.modules.pdf_interpreter import *
from src.modules.helper_fns import *

### Step 0: Initialize some presets so processing functions know how to interpret

In [3]:
'''
To assign what is considered RENT, processing functions will look for rent ranges at earlier and later regions per month.
In order to do this, you need to initialize a min/max of different rent costs that we can look for.
** Different rent ranges are really just in case of moving and a MIN/MAX in case of rent increases.
Format is as follows:
[
    (<ENTER MIN RENT FOR PROPERTY>, <ENTER MAX RENT FOR PROPERTY>),
    (...PROPERTY 2...),
    ...
]
'''

RENT_RANGES = [
    (1600, 1700),
    (600, 725)
]

### Step 1: Extract raw data from PDF bank statement

In [4]:
overall_df = generate_fin_df()

Reading PDFs from 'Ultimate Package' bucket:   0%|          | 0/40 [00:00<?, ?it/s]

Reading PDFs from 'Passport Infinite' bucket:   0%|          | 0/23 [00:00<?, ?it/s]

Reading PDFs from 'Visa' bucket:   0%|          | 0/24 [00:00<?, ?it/s]

Reading PDFs from 'MoneyMaster' bucket:   0%|          | 0/47 [00:00<?, ?it/s]

### Step 2: Process raw data based on metadata + extracted insights

In [5]:
filtered_df_2 = df_preprocessing(overall_df)
filtered_df_3 = recalibrate_amounts(filtered_df_2)
filtered_df_4 = combine_balances_across_accounts(filtered_df_3)
filtered_df_5 = tabulate_gap_balances(filtered_df_4)
filtered_df_6 = df_postprocessing(filtered_df_5, RENT_RANGES)
filtered_df_6

Savings MoneyMaster
Credit Visa
Credit Passport Infinite
Chequing Ultimate Package


,Transaction Date,Transaction Type,Amount,Balance,Details,Statement Year,Statement Month,Account Type,Account Name,Reference #,Post Date,Transaction Year,DateTime,Processed Details,Classification
0,Jan 4,Point of sale purchase,22.59,452.95,Opos Amzn Mktp CA Www.Amazon.ONCA,2021,January,Savings,MoneyMaster,NaN,NaN,2021,2021-01-04,"[opos, amzn, mktp, ca, wwwamazononca, point, o...",Online
1,Jan 4,Point of sale purchase,-4.51,448.44,Opos Amazon.ca Prime Membamazon.ca/pbcca,2021,January,Savings,MoneyMaster,NaN,NaN,2021,2021-01-04,"[opos, amazonca, prime, membamazoncapbcca, poi...",Online
2,Jan 14,Point of sale purchase,-24.96,423.48,Opos Amzn Mktp CA Www.Amazon.ONCA,2021,January,Savings,MoneyMaster,NaN,NaN,2021,2021-01-14,"[opos, amzn, mktp, ca, wwwamazononca, point, o...",Online
3,Jan 15,Point of sale purchase,-13.96,409.52,Opos Amzn Mktp CA Www.Amazon.ONCA,2021,January,Savings,MoneyMaster,NaN,NaN,2021,2021-01-15,"[opos, amzn, mktp, ca, wwwamazononca, point, o...",Online
4,Jan 15,Point of sale purchase,-13.99,395.53,Opos Amzn Mktp CA Www.Amazon.ONCA,2021,January,Savings,MoneyMaster,NaN,NaN,2021,2021-01-15,"[opos, amzn, mktp, ca, wwwamazononca, point, o...",Online
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2212,Nov 25,,-0.21,51827.54,SCOTIA CREDIT CARD PROTECTION TAX,2024,November,Credit,Passport Infinite,053,Nov 25,2024,2024-11-25,"[scotia, credit, card, protection, tax]",uncharacterized
2214,Nov 27,Withdrawal,-27.44,50916.88,32456227 Free Interac E-Transfer,2024,November,Chequing,Ultimate Package,NaN,NaN,2024,2024-11-27,"[free, interac, etransfer, withdrawal]",Reimbursement
2215,Nov 29,,-23.62,50893.26,DD/DOORDASHFAROUJKING VANCOUVER,2024,December,Credit,Visa,016,Nov 30,2024,2024-11-29,"[dddoordashfaroujking, vancouver]",Takeout
2216,Nov 29,Payroll dep.,2009.12,52926.00,Rapid Global Business Sol,2024,November,Chequing,Ultimate Package,NaN,NaN,2024,2024-11-29,"[rapid, global, business, sol, payroll, dep]",Income


### Step 3: Visualize data

#### Balance variation + insights

We visualize, apply and extract the following from the below graphs:
- Balance variation across statement transaction dates
- (Only relevant for data-processing adjustments) Various filters to assess fit to data
- Rates of changes to assess spending habits and growth of capitol

In [6]:
# def isolate_regions(peaks, troughs, zero_crossings, values):
#     regions = []

#     peaks = list(peaks)
#     troughs = list(troughs)
#     zero_crossings = list(zero_crossings)


#     # Combine all indices and sort them
#     all_points = sorted(peaks + troughs + zero_crossings)
    
#     regions = []
    
#     for i in range(len(all_points) - 1):
#         start = all_points[i]
#         end = all_points[i + 1]
        
#         if values[start] < 0:  # Below zero
#             if start in troughs or start in zero_crossings:
#                 if end in peaks:
#                     next_point = min([p for p in all_points if p > end and (p in troughs or p in zero_crossings)])
#                     regions.append((start, end, next_point))
#         else:  # Above zero
#             if start in peaks or start in zero_crossings:
#                 if end in troughs:
#                     next_point = min([p for p in all_points if p > end and (p in peaks or p in zero_crossings)])
#                     regions.append((start, end, next_point))
    
#     refined_regions = []
#     for start, mid, end in regions:
#         if values[mid] < 0:  # Below zero case
#             closest_start = max([t for t in troughs if t < mid] + [z for z in zero_crossings if z < mid])
#             closest_end = min([t for t in troughs if t > mid] + [z for z in zero_crossings if z > mid])
#         else:  # Above zero case
#             closest_start = max([p for p in peaks if p < mid] + [z for z in zero_crossings if z < mid])
#             closest_end = min([p for p in peaks if p > mid] + [z for z in zero_crossings if z > mid])
        
#         refined_regions.append((closest_start, mid, closest_end))
    
#     return refined_regions

# # Example usage:
# peaks = np.array([10, 30, 50, 70, 90])
# troughs = np.array([20, 40, 60, 80, 100])
# zero_crossings = np.array([15, 35, 55, 75, 95])
# values = np.sin(np.linspace(0, 2*np.pi, 101))  # Example values

# regions = isolate_regions(peaks, troughs, zero_crossings, values)
# print(regions)

In [7]:
# import numpy as np

# def find_regions(values, peaks, troughs, zero_crossings):
#     # Combine all indices and sort them
#     all_indices = sorted(list(peaks) + list(troughs) + list(zero_crossings))
    
#     regions = []
    
#     for i, idx in enumerate(all_indices):
#         if idx in peaks and values[idx] > 0:
#             left = max([x for x in all_indices[:i] if x in zero_crossings or x in troughs] or [0])
            
#             right = min([x for x in all_indices[i+1:] if x in zero_crossings or x in troughs] or [len(values)-1])
            
#             regions.append((left, right))
        
#         elif idx in troughs and values[idx] < 0:
#             left = max([x for x in all_indices[:i] if x in zero_crossings or x in peaks] or [0])
            
#             right = min([x for x in all_indices[i+1:] if x in zero_crossings or x in peaks] or [len(values)-1])
            
#             regions.append((left, right))
#     if 0 not in regions[0]:
#         regions.insert(0, [0,regions[0][0]])
#     if len(values)-1 not in regions[-1]:
#         regions.insert(len(regions), [regions[-1][-1], len(values)-1])
#     return regions

# # Example usage:
# values = np.array([1, 2, 1, 0, -1, -2, -1, 0, 1, 2, 1, 0, -1, -2, -1])
# peaks = np.array([1, 9])
# troughs = np.array([5, 13])
# zero_crossings = np.array([3, 7, 11])

# regions = find_regions(values, peaks, troughs, zero_crossings)
# print(regions)

In [ ]:
plot_attribute_against_datetime(
    filtered_df_6, 
    data_filtering_settings={
        'moving_average': {
            'day_span': 1,
            'gaussian_sigma': 1
        },
        'smoothed_median': {
            'window_size': 5,
        },
        'savgol_filter': {
            'window_length': 200,
            'poly_order': 5
        },
        'rate_of_change': {
            'day_span': 7,
            'gaussian_sigma': 20
        },
    },
    include_filters=True)

#### Categorical Spending Habits per Month

We visualize the following below:
- We group transactions by TRANSACTION month and then by CATEGORY
- Categories are assigned in prior code blocks; done by running through a word encoder OR by prompts for keyword associations

In [9]:
plot_stacked(filtered_df_6)

### Step 4: (Optional) View specifics of data to gain deeper insights

In [10]:
# filtered_rows = filtered_df_4[filtered_df_4['Processed Details'].apply(lambda x: any(s.lower() == 'payroll' for s in x))]

filtered_rows = filtered_df_6[filtered_df_6['Transaction Type'].apply(lambda x: x == 'Withdrawal')]
# filtered_rows = filtered_df_6[filtered_df_6['Classification'].apply(lambda x: x == 'uncharacterized')]
# filtered_rows[filtered_rows['DateTime'] >= '2023-05-01'].head(20)

# filtered_df_6[filtered_df_6['DateTime'] >= '2023-12-20'].head(20)

# filtered_rows = filtered_df_5[filtered_df_5['Details'].apply(
#     lambda x: any('airbnb' in word for word in x.lower().split())
# )]
filtered_rows.head(20)

,Transaction Date,Transaction Type,Amount,Balance,Details,Statement Year,Statement Month,Account Type,Account Name,Reference #,Post Date,Transaction Year,DateTime,Processed Details,Classification
139,Sep 10,Withdrawal,-2400.00,5421.65,80754055 Free Interac E-Transfer,2021,September,Chequing,Ultimate Package,NaN,NaN,2021,2021-09-10,"[free, interac, etransfer, withdrawal]",Reimbursement
181,Oct 2,Withdrawal,-1600.00,3070.58,76999603 Free Interac E-Transfer,2021,October,Chequing,Ultimate Package,NaN,NaN,2021,2021-10-02,"[free, interac, etransfer, withdrawal]",Rent
219,Nov 1,Withdrawal,-1600.00,4462.68,07340094 Free Interac E-Transfer,2021,November,Chequing,Ultimate Package,NaN,NaN,2021,2021-11-01,"[free, interac, etransfer, withdrawal]",Rent
221,Nov 4,Withdrawal,-239.20,4223.48,25491114 Free Interac E-Transfer,2021,November,Chequing,Ultimate Package,NaN,NaN,2021,2021-11-04,"[free, interac, etransfer, withdrawal]",Reimbursement
256,Dec 1,Withdrawal,-1600.00,5665.14,45479753 Free Interac E-Transfer,2021,December,Chequing,Ultimate Package,NaN,NaN,2021,2021-12-01,"[free, interac, etransfer, withdrawal]",Rent
284,Jan 4,Withdrawal,-1600.00,8934.68,91479386 Free Interac E-Transfer,2022,January,Chequing,Ultimate Package,NaN,NaN,2022,2022-01-04,"[free, interac, etransfer, withdrawal]",Rent
313,Feb 1,Withdrawal,-1600.00,10414.83,22773678 Free Interac E-Transfer,2022,February,Chequing,Ultimate Package,NaN,NaN,2022,2022-02-01,"[free, interac, etransfer, withdrawal]",Rent
340,Mar 1,Withdrawal,-1600.00,10697.47,48488520 Free Interac E-Transfer,2022,March,Chequing,Ultimate Package,NaN,NaN,2022,2022-03-01,"[free, interac, etransfer, withdrawal]",Rent
342,Mar 3,Withdrawal,-500.00,10179.97,59854901 Free Interac E-Transfer,2022,March,Chequing,Ultimate Package,NaN,NaN,2022,2022-03-03,"[free, interac, etransfer, withdrawal]",Reimbursement
344,Mar 4,Withdrawal,-707.39,9472.58,62279709 Free Interac E-Transfer,2022,March,Chequing,Ultimate Package,NaN,NaN,2022,2022-03-04,"[free, interac, etransfer, withdrawal]",Rent


In [11]:
# filtered_df_6[filtered_df_6['DateTime'] >= '2023-05-01'].head(40)

In [12]:
# with open('../../cached_data/databank.json', 'r') as file:
#     categories = json.load(file)

# def assign_categories(df, categories):
#     for index, row in df.iterrows():
#         details_str = ' '.join(row['Processed Details'])  # Concatenate list into a string
#         assigned = False

#         # Check for keywords in categories
#         for category, keywords in categories.items():
#             for keyword in keywords:
#                 # print(keyword)
#                 if keyword in details_str:
#                     df.at[index, 'Classification'] = category
#                     assigned = True
#                     break  # Stop checking if a keyword is found
#             if assigned:
#                 break  # Stop checking categories if one is assigned

#         # If no category was assigned, prompt the user for input
#         if not assigned:
#             print(f"Details: {row['Processed Details']}")
#             print(f"Details STR: {details_str}")
#             print("Existing categories:", list(categories.keys()))
#             print("Options for keywords (from details):", row['Processed Details'])
#             new_category = input("Enter a category to assign this: ")
#             new_keyword = input("Enter the keyword/substring identified: ")

#             # Update DataFrame and JSON data
#             df.at[index, 'Classification'] = new_category
#             if new_category in categories:
#                 categories[new_category].append(new_keyword)
#             else:
#                 categories[new_category] = [new_keyword]

#             # Remove duplicates from the list of keywords
#             categories[new_category] = list(set(categories[new_category]))

#     return df, categories

# df, updated_categories = assign_categories(filtered_df_6, categories)
# updated_categories

In [13]:
# filtered_df_6[filtered_df_6['DateTime'] >= '2021-09-20'].head(20)
# filtered_df_6[filtered_df_6['Balance'] == 6066.42].head(20)

In [14]:
# list_of_vals = list(filtered_df_4[filtered_df_4['Classification'].isnull()]['Processed Details'])
# list_of_vals

In [15]:
# filtered_df_4[filtered_df_4['Classification'].isnull()]

In [16]:
# import json
# import re

# # Load the JSON data into a dictionary
# with open('../../cached_data/databank.json', 'r') as file:
#     data = json.load(file)

# # Function to process each string
# def process_string(s):
#     # Convert to lowercase and remove special characters
#     return re.sub(r'[^a-z\s&]', '', s.lower())

# # Process each key-value pair in the dictionary
# processed_data = {key: [process_string(item) for item in value] for key, value in data.items()}
# with open('../../cached_data/databank.json', 'w') as file:
#     json.dump(processed_data, file, indent=4)

In [17]:
# import pandas as pd
# import plotly.graph_objects as go

# fig = go.Figure()
# fig.add_trace(go.Scatter(x=filtered_df['DateTime'], y=filtered_df['Amount'], mode='lines+markers', name='Amount'))
# fig.update_layout(
#     title='Amount Over Time',
#     xaxis_title='Date',
#     yaxis_title='Amount',
#     xaxis=dict(
#         tickformat='%Y-%m-%d',  # Format for x-axis labels
#         dtick='D1'              # Set tick interval to one day
#     )
# )

# fig.show()


In [18]:
def filter_string(input_string):
    elements = re.split(r'[ _*-]+', input_string)
    return [elem for elem in elements if not any(char.isdigit() for char in elem)]

input_str = filtered_df['Details'].iloc[1]
filter_string(input_str)

NameError: name 'filtered_df' is not defined

In [7]:
# def extract_features_from_pdf(pdf_file, x_tolerance=2, init_y_top=440, reg_y_top=210):

#     parent_dir = os.path.dirname(pdf_file)
#     dir_before_file = os.path.basename(parent_dir)
#     if dir_before_file == "Chequing":
#         x_right = 600
#         x_left = 750
#         init_y_top = 400
#         regular_page_box = (70, reg_y_top, x_right, x_left)
#         initial_page_box = (70, init_y_top, x_right, x_left)
#     elif dir_before_file == "Credit":
#         x_right = 400
#         x_left = 730
#         reg_y_top = 100
#         init_y_top = 600
#         regular_page_box = (70, reg_y_top, x_right, x_left)
#         initial_page_box = (70, init_y_top, x_right, x_left)

#     text = ''
#     # Open the PDF file
#     with pdfplumber.open(pdf_file) as pdf:
#         for page in pdf.pages:
#             if page.page_number == 1:
#                 rect = initial_page_box
#             else:
#                 rect = regular_page_box

#             # Extract text only from the specified rectangle
#             text += page.crop(rect).extract_text(x_tolerance=x_tolerance)

#     # Process the extracted text
#     lines = text.split('\n')
#     return lines

In [ ]:
def __grab_pattern(account_type):
    if account_type == "Credit":
        return r'(\d{3})\s+(\w{3}\s+\d{1,2})\s+(\w{3}\s+\d{1,2})\s+(.+)\s+(\d+\.\d{2})(-)?'
    elif account_type in ["Chequing", "Savings"]:
        return r'(\w+ \d+)\s+(.*?)\s+(\d{1,3}(?:,\d{3})*(?:\.\d{2})?)\s+(\d{1,3}(?:,\d{3})*(?:\.\d{2})?)'


def process_transactions_from_lines(pdf_lines, account_type):

    transactions = []

    pattern = __grab_pattern(account_type)

    for i, line in enumerate(pdf_lines):
        match = re.match(pattern, line)
        # print(line)
        if match:
            match_groups = list(match.groups())
            print(match_groups)
            if account_type == "Credit":
                ref_num = match_groups[0]
                transaction_date = match_groups[1]
                post_date = match_groups[2]
                details = match_groups[3]
                amount = match_groups[4]
                # Condition where credit statement indicates this was a deposit, so we see a "-" at end of amount
                if match_groups[5] == "-":
                    amount = str(float(amount) * -1)
                # Create a dictionary for the transaction and add it to the list
                transaction = {
                    'Reference #': ref_num,
                    'Transaction Date': transaction_date,
                    'Post Date': post_date,
                    'Details': details,
                    'Amount': amount,
                    'Transaction Type': '',
                }
            elif account_type in ["Chequing", "Savings"]:
                date = match_groups[0]
                transaction_type = match_groups[1]
                if match_groups[3] is None:
                    amount = None
                    balance = match_groups[2]
                else:
                    amount = match_groups[2]
                    balance = match_groups[3]

                # Create a dictionary for the transaction and add it to the list
                transaction = {
                    'Transaction Date': date,
                    'Transaction Type': transaction_type,
                    'Amount': amount,
                    'Balance': balance,
                    'Details': pdf_lines[i+1],
                }
            transactions.append(transaction)

    return transactions

# pdf_file = "../../bank_statements/Chequing/Ultimate Package/April 2024 e-statement.pdf"
pdf_file = "../../bank_statements/Credit/Visa/June 2023 e-statement.pdf"
lines = extract_features_from_pdf(pdf_file, x_tolerance=2, init_y_top=440, reg_y_top=210)
transactions = process_transactions_from_lines(lines, "Credit")
# transactions

['002', 'May 1', 'May 3', 'REXALL PHARMACY #1347 MISSISSAUGA ON', '27.11', None]
['003', 'May 2', 'May 3', '1859-TOR 6 & SUNDRY T3 MISSISSAUGAON', '15.64', None]
['004', 'May 3', 'May 3', 'MB-CREDIT CARD/LOC PAY. FROM -', '99.98', '-']
['005', 'May 4', 'May 4', 'MB-CREDIT CARD/LOC PAY. FROM -', '42.75', '-']
['006', 'May 8', 'May 9', 'trainline +443332022222', '269.70', None]
['007', 'May 9', 'May 9', 'CHATGPT SUBSCRIPTION OPENAI.COMCAAMT', '31.05', None]
['008', 'May 10', 'May 10', 'MB-CREDIT CARD/LOC PAY. FROM -', '300.75', '-']
['010', 'May 11', 'May 15', "ALI 'S WEST INDIAN ROTITORONTO ON", '17.80', None]
['011', 'May 11', 'May 15', "ALI 'S WEST INDIAN ROTITORONTO ON", '22.37', None]
['012', 'May 12', 'May 15', 'THE DANISH PASTRY HOUSETORONTOON', '2.73', None]
['013', 'May 14', 'May 15', 'AMZN Mktp US*X08R39PM3Amzn.com/billON', '58.01', None]
['014', 'May 15', 'May 16', 'A&W 4965 NEPEANON', '11.16', None]
['015', 'May 15', 'May 16', 'OCT-CONFEDERATION-PARLIAM OTTAWAON', '20.00', No

In [39]:
lines

['SUB-TOTAL CREDITS $0.00',
 'SUB-TOTAL DEBITS $29.00',
 'MR JAVAID A BAKSH -4537 344 247 272012',
 '002 May 1 May 3 REXALL PHARMACY #1347 MISSISSAUGA ON 27.11',
 '003 May 2 May 3 1859-TOR 6 & SUNDRY T3 MISSISSAUGAON 15.64',
 '004 May 3 May 3 MB-CREDIT CARD/LOC PAY. FROM - 99.98-',
 '*****04*1181',
 '005 May 4 May 4 MB-CREDIT CARD/LOC PAY. FROM - 42.75-',
 '*****04*1181',
 '006 May 8 May 9 trainline +443332022222 269.70',
 '007 May 9 May 9 CHATGPT SUBSCRIPTION OPENAI.COMCAAMT 31.05',
 '22.60 UNITED STATES DOLLAR',
 '008 May 10 May 10 MB-CREDIT CARD/LOC PAY. FROM - 300.75-',
 '*****04*1181ON ABOUT YOUR SCOTIABANK STATEMENT',
 'statement carefully: If you believe there is an error, contact us through For Scotiabank Visa c',
 'r at any number listed on this statementwithin 15 days of your statement Card Cheques: The e',
 'ise it is considered correct (except any item or credit posted in error). charged to customer',
 'ant Information: This statement is a summary only. Review the Transacti

In [46]:
transactions

[{'Reference #': '002',
  'Transaction Date': 'May 1',
  'Post Date': 'May 3',
  'Details': 'REXALL PHARMACY #1347 MISSISSAUGA ON',
  'Amount': '27.11',
  'Transaction Type': ''},
 {'Reference #': '003',
  'Transaction Date': 'May 2',
  'Post Date': 'May 3',
  'Details': '1859-TOR 6 & SUNDRY T3 MISSISSAUGAON',
  'Amount': '15.64',
  'Transaction Type': ''},
 {'Reference #': '004',
  'Transaction Date': 'May 3',
  'Post Date': 'May 3',
  'Details': 'MB-CREDIT CARD/LOC PAY. FROM -',
  'Amount': '-99.98',
  'Transaction Type': ''},
 {'Reference #': '005',
  'Transaction Date': 'May 4',
  'Post Date': 'May 4',
  'Details': 'MB-CREDIT CARD/LOC PAY. FROM -',
  'Amount': '-42.75',
  'Transaction Type': ''},
 {'Reference #': '006',
  'Transaction Date': 'May 8',
  'Post Date': 'May 9',
  'Details': 'trainline +443332022222',
  'Amount': '269.70',
  'Transaction Type': ''},
 {'Reference #': '007',
  'Transaction Date': 'May 9',
  'Post Date': 'May 9',
  'Details': 'CHATGPT SUBSCRIPTION OPENAI.C

In [191]:
text =  'Apr 5 Payroll dep. 1,178.95 36,948.98'
pattern = r'(\w+ \d+)\s+(.*?)\s+(\d{1,3}(?:,\d{3})*(?:\.\d{2})?)\s+(\d{1,3}(?:,\d{3})*(?:\.\d{2})?)'
match = re.match(pattern, text)
if match:
    date = match.group(1)          # 'Apr 1'
    description = match.group(2)   # 'Value towards sdf'
    value = match.group(3)         # '25.00'
    balance = match.group(4)       # '37,434,123.39'

    print(f"Date: {date}")
    print(f"Description: {description}")
    print(f"Value: {value}")
    print(f"Balance: {balance}")
else:
    print("No match found.")
    

Date: Apr 5
Description: Payroll dep.
Value: 1,178.95
Balance: 36,948.98
